In [2]:
%load_ext memory_profiler
# !pip install -q zhconv

In [3]:
import os 

# Packages
import gensim
import jieba
import zhconv
from gensim.corpora import WikiCorpus
from datetime import datetime as dt
from typing import List


if not os.path.isfile('dict.txt.big'):
    !wget https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big
jieba.set_dictionary('dict.txt.big')

print("gensim", gensim.__version__)
print("jieba", jieba.__version__)

gensim 4.3.1
jieba 0.42.1


In [9]:
!wget "https://dumps.wikimedia.org/zhwiki/20230501/zhwiki-20230501-pages-articles-multistream.xml.bz2"

--2023-05-08 19:44:19--  https://dumps.wikimedia.org/zhwiki/20230501/zhwiki-20230501-pages-articles-multistream.xml.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 2620:0:861:2:208:80:154:142, 208.80.154.142
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|2620:0:861:2:208:80:154:142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2790968986 (2.6G) [application/octet-stream]
Saving to: 'zhwiki-20230501-pages-articles-multistream.xml.bz2.1'

zhwiki-20230501-pag 100%[===================>]   2.60G  3.76MB/s    in 10m 56s 

2023-05-08 19:55:17 (4.06 MB/s) - 'zhwiki-20230501-pages-articles-multistream.xml.bz2.1' saved [2790968986/2790968986]



In [ ]:
# save these into wiki_to_txt.py
# -*- coding: utf-8 -*-

import logging
import sys

from gensim.corpora import WikiCorpus

def main():

    if len(sys.argv) != 2:
        print("Usage: python3 " + sys.argv[0] + " wiki_data_path")
        exit()

    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
    wiki_corpus = WikiCorpus(sys.argv[1], dictionary={})
    texts_num = 0

    with open("wiki_texts.txt",'w',encoding='utf-8') as output:
        for text in wiki_corpus.get_texts():
            output.write(' '.join(text) + '\n')
            texts_num += 1
            if texts_num % 10000 == 0:
                logging.info("已處理 %d 篇文章" % texts_num)

if __name__ == "__main__":
    main()

In [ ]:
# run 'python3 wiki_to_txt.py zhwiki-20230501-pages-articles-multistream.xml.bz2' in shell


In [ ]:
# run 'opencc -i wiki_texts.txt -o wiki_zh_tw.txt -c s2tw.json' in terminal in shell 
# 將簡體轉成繁體


In [1]:
import jieba
import logging

def main():

    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

    # jieba custom setting.
    jieba.set_dictionary('dict.txt.big')

    # load stopwords set
    stopword_set = set()
    with open('stopword.txt','r', encoding='utf-8') as stopwords:
        for stopword in stopwords:
            stopword_set.add(stopword.strip('\n'))

    output = open('wiki_seg.txt', 'w', encoding='utf-8')
    with open('wiki_zh_tw.txt', 'r', encoding='utf-8') as content :
        for texts_num, line in enumerate(content):
            line = line.strip('\n')
            words = jieba.cut(line, cut_all=False)
            for word in words:
                if word not in stopword_set:
                    output.write(word + ' ')
            output.write('\n')

            if (texts_num + 1) % 10000 == 0:
                logging.info("已完成前 %d 行的斷詞" % (texts_num + 1))
    output.close()

if __name__ == '__main__':
    main()


Building prefix dict from /Users/linli/Desktop/hw4/dict.txt.big ...
2023-05-10 00:48:28,870 : DEBUG : Building prefix dict from /Users/linli/Desktop/hw4/dict.txt.big ...
Loading model from cache /var/folders/hd/b9ptqs9s7wgd9115fq_x_b100000gn/T/jieba.ufe8439f3956c82211e6ef529fb07e2bb.cache
2023-05-10 00:48:28,874 : DEBUG : Loading model from cache /var/folders/hd/b9ptqs9s7wgd9115fq_x_b100000gn/T/jieba.ufe8439f3956c82211e6ef529fb07e2bb.cache
Loading model cost 2.055 seconds.
2023-05-10 00:48:30,929 : DEBUG : Loading model cost 2.055 seconds.
Prefix dict has been built successfully.
2023-05-10 00:48:30,935 : DEBUG : Prefix dict has been built successfully.
2023-05-10 00:53:48,012 : INFO : 已完成前 10000 行的斷詞
2023-05-10 00:57:51,060 : INFO : 已完成前 20000 行的斷詞
2023-05-10 01:01:42,247 : INFO : 已完成前 30000 行的斷詞
2023-05-10 01:05:07,020 : INFO : 已完成前 40000 行的斷詞
2023-05-10 01:08:40,231 : INFO : 已完成前 50000 行的斷詞
2023-05-10 01:11:57,223 : INFO : 已完成前 60000 行的斷詞
2023-05-10 01:15:20,311 : INFO : 已完成前 70000 

In [18]:
import logging

from gensim.models import word2vec

# def main():

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
sentences = word2vec.LineSentence("wiki_seg.txt")
model = word2vec.Word2Vec(sentences, vector_size=250)

#保存模型，供日後使用
model.save("word2vec.model")

#模型讀取方式
# model = word2vec.Word2Vec.load("your_model_name")

# if __name__ == "__main__":
#     main()

2023-05-10 14:34:02,137 : INFO : collecting all words and their counts
2023-05-10 14:34:02,142 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2023-05-10 14:34:06,076 : INFO : PROGRESS: at sentence #10000, processed 9911456 words, keeping 621212 word types
2023-05-10 14:34:08,923 : INFO : PROGRESS: at sentence #20000, processed 17556176 words, keeping 901885 word types
2023-05-10 14:34:11,669 : INFO : PROGRESS: at sentence #30000, processed 24317345 words, keeping 1106317 word types
2023-05-10 14:34:14,130 : INFO : PROGRESS: at sentence #40000, processed 30709892 words, keeping 1284863 word types
2023-05-10 14:34:16,438 : INFO : PROGRESS: at sentence #50000, processed 36665234 words, keeping 1435747 word types
2023-05-10 14:34:18,490 : INFO : PROGRESS: at sentence #60000, processed 42186808 words, keeping 1572479 word types
2023-05-10 14:34:20,545 : INFO : PROGRESS: at sentence #70000, processed 47613578 words, keeping 1694482 word types
2023-05-10 14:34:22,3

2023-05-10 14:35:46,157 : INFO : EPOCH 0 - PROGRESS: at 0.54% examples, 631708 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:35:47,160 : INFO : EPOCH 0 - PROGRESS: at 0.72% examples, 655452 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:35:48,173 : INFO : EPOCH 0 - PROGRESS: at 0.94% examples, 656461 words/s, in_qsize 5, out_qsize 1
2023-05-10 14:35:49,179 : INFO : EPOCH 0 - PROGRESS: at 1.09% examples, 654731 words/s, in_qsize 4, out_qsize 1
2023-05-10 14:35:50,189 : INFO : EPOCH 0 - PROGRESS: at 1.25% examples, 659586 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:35:51,195 : INFO : EPOCH 0 - PROGRESS: at 1.45% examples, 666250 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:35:52,216 : INFO : EPOCH 0 - PROGRESS: at 1.63% examples, 669783 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:35:53,222 : INFO : EPOCH 0 - PROGRESS: at 1.82% examples, 673689 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:35:54,228 : INFO : EPOCH 0 - PROGRESS: at 2.00% examples, 672022 words/s, in_qsize 5, out_

2023-05-10 14:36:59,894 : INFO : EPOCH 0 - PROGRESS: at 19.07% examples, 689644 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:37:00,898 : INFO : EPOCH 0 - PROGRESS: at 19.40% examples, 689534 words/s, in_qsize 6, out_qsize 0
2023-05-10 14:37:01,905 : INFO : EPOCH 0 - PROGRESS: at 19.72% examples, 689242 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:37:02,909 : INFO : EPOCH 0 - PROGRESS: at 20.09% examples, 689898 words/s, in_qsize 6, out_qsize 0
2023-05-10 14:37:03,912 : INFO : EPOCH 0 - PROGRESS: at 20.46% examples, 690792 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:37:04,925 : INFO : EPOCH 0 - PROGRESS: at 20.82% examples, 691703 words/s, in_qsize 6, out_qsize 0
2023-05-10 14:37:05,928 : INFO : EPOCH 0 - PROGRESS: at 21.17% examples, 692644 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:37:06,935 : INFO : EPOCH 0 - PROGRESS: at 21.51% examples, 693315 words/s, in_qsize 5, out_qsize 1
2023-05-10 14:37:07,949 : INFO : EPOCH 0 - PROGRESS: at 21.88% examples, 694257 words/s, in_qsiz

2023-05-10 14:38:13,510 : INFO : EPOCH 0 - PROGRESS: at 47.68% examples, 731125 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:38:14,532 : INFO : EPOCH 0 - PROGRESS: at 48.14% examples, 731345 words/s, in_qsize 6, out_qsize 0
2023-05-10 14:38:15,533 : INFO : EPOCH 0 - PROGRESS: at 48.55% examples, 731731 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:38:16,535 : INFO : EPOCH 0 - PROGRESS: at 48.99% examples, 731959 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:38:17,546 : INFO : EPOCH 0 - PROGRESS: at 49.49% examples, 732344 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:38:18,576 : INFO : EPOCH 0 - PROGRESS: at 49.92% examples, 732551 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:38:19,582 : INFO : EPOCH 0 - PROGRESS: at 50.32% examples, 733900 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:38:20,593 : INFO : EPOCH 0 - PROGRESS: at 50.76% examples, 734224 words/s, in_qsize 5, out_qsize 1
2023-05-10 14:38:21,593 : INFO : EPOCH 0 - PROGRESS: at 51.22% examples, 734668 words/s, in_qsiz

2023-05-10 14:39:27,144 : INFO : EPOCH 0 - PROGRESS: at 78.32% examples, 748013 words/s, in_qsize 6, out_qsize 0
2023-05-10 14:39:28,144 : INFO : EPOCH 0 - PROGRESS: at 78.64% examples, 747385 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:39:29,165 : INFO : EPOCH 0 - PROGRESS: at 79.06% examples, 747507 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:39:30,177 : INFO : EPOCH 0 - PROGRESS: at 79.46% examples, 747544 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:39:31,186 : INFO : EPOCH 0 - PROGRESS: at 79.92% examples, 747719 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:39:32,191 : INFO : EPOCH 0 - PROGRESS: at 80.35% examples, 747839 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:39:33,195 : INFO : EPOCH 0 - PROGRESS: at 80.78% examples, 747977 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:39:34,202 : INFO : EPOCH 0 - PROGRESS: at 81.20% examples, 748119 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:39:35,207 : INFO : EPOCH 0 - PROGRESS: at 81.61% examples, 748244 words/s, in_qsiz

...

2023-05-10 14:52:54,391 : INFO : EPOCH 3 - PROGRESS: at 79.62% examples, 770258 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:52:55,407 : INFO : EPOCH 3 - PROGRESS: at 80.08% examples, 770327 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:52:56,411 : INFO : EPOCH 3 - PROGRESS: at 80.53% examples, 770475 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:52:57,415 : INFO : EPOCH 3 - PROGRESS: at 80.95% examples, 770525 words/s, in_qsize 6, out_qsize 0
2023-05-10 14:52:58,417 : INFO : EPOCH 3 - PROGRESS: at 81.37% examples, 770598 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:52:59,425 : INFO : EPOCH 3 - PROGRESS: at 81.77% examples, 770592 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:53:00,429 : INFO : EPOCH 3 - PROGRESS: at 82.14% examples, 770679 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:53:01,443 : INFO : EPOCH 3 - PROGRESS: at 82.54% examples, 770658 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:53:02,452 : INFO : EPOCH 3 - PROGRESS: at 82.95% examples, 770674 words/s, in_qsiz

2023-05-10 14:54:07,488 : INFO : EPOCH 4 - PROGRESS: at 5.57% examples, 759459 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:54:08,493 : INFO : EPOCH 4 - PROGRESS: at 5.80% examples, 758617 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:54:09,503 : INFO : EPOCH 4 - PROGRESS: at 6.06% examples, 757631 words/s, in_qsize 6, out_qsize 0
2023-05-10 14:54:10,516 : INFO : EPOCH 4 - PROGRESS: at 6.33% examples, 756380 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:54:11,517 : INFO : EPOCH 4 - PROGRESS: at 6.59% examples, 753543 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:54:12,519 : INFO : EPOCH 4 - PROGRESS: at 6.81% examples, 751245 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:54:13,531 : INFO : EPOCH 4 - PROGRESS: at 7.04% examples, 750286 words/s, in_qsize 4, out_qsize 1
2023-05-10 14:54:14,545 : INFO : EPOCH 4 - PROGRESS: at 7.30% examples, 749525 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:54:15,556 : INFO : EPOCH 4 - PROGRESS: at 7.57% examples, 749441 words/s, in_qsize 6, out_

2023-05-10 14:55:21,078 : INFO : EPOCH 4 - PROGRESS: at 30.08% examples, 765731 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:55:22,091 : INFO : EPOCH 4 - PROGRESS: at 30.45% examples, 765951 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:55:23,101 : INFO : EPOCH 4 - PROGRESS: at 30.84% examples, 766003 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:55:24,113 : INFO : EPOCH 4 - PROGRESS: at 31.24% examples, 766270 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:55:25,118 : INFO : EPOCH 4 - PROGRESS: at 31.58% examples, 766416 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:55:26,129 : INFO : EPOCH 4 - PROGRESS: at 31.97% examples, 766585 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:55:27,140 : INFO : EPOCH 4 - PROGRESS: at 32.37% examples, 766578 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:55:28,154 : INFO : EPOCH 4 - PROGRESS: at 32.79% examples, 766696 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:55:29,172 : INFO : EPOCH 4 - PROGRESS: at 33.20% examples, 766639 words/s, in_qsiz

2023-05-10 14:56:34,705 : INFO : EPOCH 4 - PROGRESS: at 58.24% examples, 753034 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:56:35,714 : INFO : EPOCH 4 - PROGRESS: at 58.60% examples, 752777 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:56:36,720 : INFO : EPOCH 4 - PROGRESS: at 58.93% examples, 752507 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:56:37,746 : INFO : EPOCH 4 - PROGRESS: at 59.29% examples, 752348 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:56:38,749 : INFO : EPOCH 4 - PROGRESS: at 59.60% examples, 751760 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:56:39,758 : INFO : EPOCH 4 - PROGRESS: at 59.94% examples, 751174 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:56:40,759 : INFO : EPOCH 4 - PROGRESS: at 60.33% examples, 751197 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:56:42,175 : INFO : EPOCH 4 - PROGRESS: at 60.63% examples, 748777 words/s, in_qsize 4, out_qsize 2
2023-05-10 14:56:43,196 : INFO : EPOCH 4 - PROGRESS: at 60.78% examples, 746092 words/s, in_qsiz

2023-05-10 14:57:48,835 : INFO : EPOCH 4 - PROGRESS: at 84.87% examples, 729961 words/s, in_qsize 6, out_qsize 0
2023-05-10 14:57:49,840 : INFO : EPOCH 4 - PROGRESS: at 85.25% examples, 730130 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:57:50,856 : INFO : EPOCH 4 - PROGRESS: at 85.70% examples, 730312 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:57:51,867 : INFO : EPOCH 4 - PROGRESS: at 86.14% examples, 730376 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:57:52,873 : INFO : EPOCH 4 - PROGRESS: at 86.57% examples, 730529 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:57:53,873 : INFO : EPOCH 4 - PROGRESS: at 87.01% examples, 730680 words/s, in_qsize 5, out_qsize 0
2023-05-10 14:57:54,887 : INFO : EPOCH 4 - PROGRESS: at 87.48% examples, 730805 words/s, in_qsize 6, out_qsize 0
2023-05-10 14:57:55,897 : INFO : EPOCH 4 - PROGRESS: at 87.89% examples, 730983 words/s, in_qsize 6, out_qsize 0
2023-05-10 14:57:56,898 : INFO : EPOCH 4 - PROGRESS: at 88.35% examples, 731179 words/s, in_qsiz

In [40]:
! ls word2vec*

word2vec.model                word2vec.model.wv.vectors.npy
word2vec.model.syn1neg.npy


In [20]:
print(model.wv.vectors.shape)
model.wv.vectors

(1003966, 250)


array([[-1.63678801e+00,  1.28177965e+00, -3.98507565e-01, ...,
        -1.42127526e+00,  6.82937682e-01, -4.20246750e-01],
       [-2.97019148e+00, -9.38789666e-01, -1.73306882e+00, ...,
        -1.05096638e+00, -5.96617818e-01,  2.04739308e+00],
       [-9.44583595e-01, -4.66692150e-01, -9.01146054e-01, ...,
        -2.93332911e+00, -1.90525711e-01,  1.46472037e-01],
       ...,
       [-3.39438394e-02, -9.94087160e-02, -1.20609745e-01, ...,
         4.12042290e-02,  2.69078594e-02,  2.37404890e-02],
       [ 1.21654339e-01,  1.57428980e-02,  5.76299205e-02, ...,
         1.35978572e-02,  9.91761312e-03,  5.99864982e-02],
       [ 4.36115339e-02,  4.37661186e-02,  5.42736053e-02, ...,
         1.40509824e-03,  1.43255796e-02,  2.78084576e-02]], dtype=float32)

In [24]:
# print(f"總共收錄了 {len(model.wv.vocab)} 個詞彙")

print("印出 20 個收錄詞彙:")
print(list(model.wv.key_to_index.keys())[:20])


印出 20 個收錄詞彙:
['年', '中', '日', '小行星', '月', '中國', '日本', '一個', '時', '香港', '美國', '陳', '臺灣', '站', 'zh', 'px', '研究', '新', '電影', '外部']


In [25]:
vec = model.wv['數學家']
print(vec.shape)
vec 

(250,)


array([-4.12086278e-01,  4.99083288e-02, -3.72508019e-01,  8.75706673e-01,
       -8.31748843e-01, -1.16827023e+00,  2.10350561e+00, -4.95293140e-01,
       -1.38021719e+00, -5.69052815e-01,  1.90268293e-01,  1.53564110e-01,
       -9.98773098e-01, -1.92481923e+00,  1.06128705e+00,  8.16954017e-01,
       -1.25862372e+00,  1.25781095e+00,  1.97392857e+00,  1.74641657e+00,
        2.40567970e+00,  8.78834426e-01, -1.14328839e-01,  1.30477870e+00,
        1.31969368e+00, -2.78753657e-02, -6.73492253e-02,  1.40664887e+00,
       -1.33565497e+00, -5.10471642e-01, -4.82397914e-01,  8.55136812e-01,
       -1.05795729e+00, -3.17323148e-01,  1.57992232e+00, -1.56151843e+00,
       -6.61998391e-01,  1.31999505e+00,  2.72087193e+00, -1.19542524e-01,
       -4.42690372e+00,  1.38626974e-02,  2.07557249e+00,  3.95296186e-01,
        2.47598505e+00,  1.62376201e+00,  7.98167169e-01,  4.58470821e-01,
       -1.60242856e-01,  2.54063725e-01,  5.14172554e-01,  8.90175581e-01,
        2.37658763e+00, -

In [26]:
word = "這肯定沒見過 "

# 若強行取值會報錯
try:
    vec = model.wv[word]
except KeyError as e:
    print(e)


"Key '這肯定沒見過 ' not present"


In [27]:
model.wv.most_similar("飲料", topn=10)

[('飲品', 0.8193895220756531),
 ('果汁', 0.7639946341514587),
 ('含酒精', 0.7008175253868103),
 ('酒類', 0.697655200958252),
 ('罐裝', 0.686590313911438),
 ('口香糖', 0.6708953380584717),
 ('冰淇淋', 0.6650984287261963),
 ('可口可樂', 0.6629427671432495),
 ('酸奶', 0.6596047282218933),
 ('蘇打水', 0.6542328000068665)]

In [28]:
model.wv.most_similar("car")

[('tikita', 0.7003545165061951),
 ('truck', 0.6954656839370728),
 ('motor', 0.6678410768508911),
 ('seat', 0.6515442132949829),
 ('cab', 0.6467195153236389),
 ('wagon', 0.6306233406066895),
 ('automobile', 0.6303455829620361),
 ('videna', 0.6291805505752563),
 ('motorcycle', 0.6260409951210022),
 ('vehicle', 0.622925341129303)]

In [29]:
model.wv.most_similar("facebook")

[('專頁', 0.7808370590209961),
 ('臉書', 0.7553152441978455),
 ('instagram', 0.7440596222877502),
 ('粉絲團', 0.6917836666107178),
 ('粉專', 0.6887847185134888),
 ('blogger', 0.672474205493927),
 ('面書', 0.670678973197937),
 ('討論區', 0.6646865606307983),
 ('貼文', 0.6617392897605896),
 ('tumblr', 0.6465764045715332)]

In [30]:
model.wv.most_similar("詐欺")

[('欺詐', 0.7395585179328918),
 ('詐騙', 0.6012449264526367),
 ('慣犯', 0.5685928463935852),
 ('詐欺罪', 0.5661442279815674),
 ('前科', 0.5379853844642639),
 ('被害者', 0.5319794416427612),
 ('竊盜', 0.5294862985610962),
 ('信用調查', 0.5253278613090515),
 ('作假', 0.523360550403595),
 ('惡質', 0.5226569771766663)]

In [31]:
model.wv.most_similar("合約")

[('合同', 0.7713289856910706),
 ('簽約', 0.6876683831214905),
 ('續約', 0.6701698899269104),
 ('租約', 0.6425994038581848),
 ('簽下', 0.6315926313400269),
 ('續簽', 0.6314466595649719),
 ('選擇權', 0.6119654774665833),
 ('買斷', 0.5992292165756226),
 ('解約', 0.5787010192871094),
 ('新東家', 0.5755165219306946)]

In [32]:
model.wv.similarity("連結", "鍵接")

0.7381378

In [33]:
model.wv.similarity("連結", "陰天")

0.007539022

In [38]:
# print(f"Loading {output_model}...")
# print (f"word2vec.zh.{word_dim_size}.model")
new_model = word2vec.Word2Vec.load(f"word2vec.model")

2023-05-10 21:48:15,543 : INFO : loading Word2Vec object from word2vec.model
2023-05-10 21:48:16,044 : INFO : loading wv recursively from word2vec.model.wv.* with mmap=None
2023-05-10 21:48:16,045 : INFO : loading vectors from word2vec.model.wv.vectors.npy with mmap=None
2023-05-10 21:48:16,720 : INFO : loading syn1neg from word2vec.model.syn1neg.npy with mmap=None
2023-05-10 21:48:17,556 : INFO : setting ignored attribute cum_table to None
2023-05-10 21:48:25,174 : INFO : Word2Vec lifecycle event {'fname': 'word2vec.model', 'datetime': '2023-05-10T21:48:25.174600', 'gensim': '4.3.1', 'python': '3.9.13 (main, Aug 25 2022, 18:29:29) \n[Clang 12.0.0 ]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'loaded'}


In [39]:
model.wv.similarity("連結", "陰天") == new_model.wv.similarity("連結", "陰天")

True